# tools

> To analyze the deep sequencing data in pandas.

In [ ]:
#| default_exp tools

#| export
import logging
import sys
import pandas as pd

def _to_df(
    read,
    )->pd.DataFrame:
    """
    Alignment to dataframe.

    Parameters:
        read : read object of pysam.

    Return: dataframe (pd.DataFrame)
    """
    ## Make a DataFrame
    return (pd.DataFrame(
         read.get_aligned_pairs(with_seq=True),
         columns=['read position','reference position','reference'],
         )
     .assign(
         read=lambda df: df['read position'].map(
         ## Get all the reference (position) to query mappings
         dict(zip(range(read.query_alignment_length),
                 list(read.query_alignment_sequence)))
            )))

def read_bam(
    bam_path: str,
    verbose: bool=False,
    force: bool= False,
    ):
    """
    Read the aligned file

    Parameters:
        bam_path (str): path to the .bam file.

    Returns: pysam's aligned object.
    """
    ## index the bam file. makes .bai file
    from pathlib import Path
    import pysam

    if not Path(bam_path+'.bai').exists() or force:
        _=pysam.index(bam_path)
    aligned=pysam.AlignmentFile(bam_path, 'rb')
    if verbose: logging.info(f"size of the bam file = "+f"{round(sys.getsizeof(aligned)/(pow(1024,3)), 2)} GB")    
    return aligned


def get_reads_per_contig(
    aligned,
    plot: bool=False,
    ) -> pd.DataFrame:
    """
    Cumulative read depth per contigs.

    Parameters:
        aliigned: aligned object of pysam.
        plot (bool): make a plot.

    Returns:
        pd.DataFrame
    """
    def get_attrs(o):
        return {k:getattr(o,k) for k in dir(o) if k in ['contig','mapped','total','unmapped']}
    df0=(
        pd.DataFrame(map(get_attrs,aligned.get_index_statistics()))
        .query("`mapped`!=0")
        .sort_values('mapped',ascending=False)
        )
    _=df0.plot.barh(x='contig',y=['mapped','unmapped'],stacked=True,ylabel='reads')
    return df0

def to_df(
    bam_path: str,
    start: int,
    end: int,
    contig: str,
    verbose: bool=False,
    **kws_fetch,
    ) -> pd.DataFrame:
    """
    Make a dataframe from a bam file.

    Parameters:
        bam_path (str): path to the bam file.
        start (int): start position.
        end (int): end position.
        contig (str): contig name.
        verbose (bool): verbose.

    Keyword parameters:
        kws_fetch: parameters provided to `aligned.fetch` function of pysam.

    Returns:
        pd.DataFrame
    """
    aligned=read_bam(
        bam_path,
        verbose=verbose,
    )
    ## Filter the reads by target i.e. contig, start and end
    ## docs: https://pysam.readthedocs.io/en/latest/api.html#pysam.AlignmentFile.fetch
    aligned_reads = aligned.fetch(
        start=start,
        end=end,
        contig=contig,
        **kws_fetch,
    )
    ## concatenate the dataframes of reads
    df1=(
        pd.concat(
            {(read.qname,read.cigarstring):_to_df(read) for read in aligned_reads},
            axis=0,
            names=['query id','CIGAR'],
            )
        .reset_index([0,1])
        .assign(
            reference= lambda x: x['reference'].str.upper(),
            read= lambda x: x['read'].str.upper(),
            )
        .query(expr=f"`reference position`>={start} and `reference position`<={end}")
        )
    assert not df1.duplicated().any()
    if verbose: logging.info(f"size of the table = "+f"{round(sys.getsizeof(df1)/(pow(1024,3)), 2)} GB")
    return df1

def mask_wt(
    df3: pd.DataFrame,
    values: str,
    columns: list=['reference','read'],
    ) -> pd.DataFrame:
    """
    Mask the values for WT i.e. reference==read.

    Parameters:
        values (str): column with values.
        columns (list): the columns to check the WT in. defaults to ['reference','read'].

    Returns:
        pd.DataFrame
    """
    return (
        df3
        .assign(
        **{values:lambda df: df.apply(lambda x: None if x[columns[0]]==x[columns[1]] else x[values],axis=1),}
        )
    )

@pd.api.extensions.register_dataframe_accessor("dseq")
class Dseq:
    """Class for operating on the dataframe created from the .bam file."""
    def __init__(self, pandas_obj):
        self._obj = pandas_obj

    # @property
    # def name(self): return

    def get_coverage(
        self
        ) -> pd.DataFrame:
        """
        Get the coverage by reference position.
        """
        return (
            self._obj
            .groupby(['reference position',"reference"]).size().to_frame('depth').reset_index()
        )

    def get_counts_per_mutation(
        self,
        ) -> pd.DataFrame:
        """
        Get number of reads per counts of the mutations.
        """
        return (self._obj
            .query("`reference`!=`read`") ## filter to keep mutations
            .groupby('query id').size()   ## count mutations by reads
            .value_counts()               ## count reads by count mutations
            .sort_values(ascending=True)  ## format data for plot
        )
    def get_counts_per_nt_mutation(
        self
        ) -> pd.DataFrame:
        """
        Get counts of the nucleotide mutations.
        """
        return (self._obj
            .query("~(`CIGAR`.str.contains('I') or `CIGAR`.str.contains('D'))") ## remove indels
            .groupby(['reference position',"reference","read"]).size()          ## depth of mutations
            .to_frame('depth').reset_index()
            .pipe(mask_wt,'depth')                                              ## mask wt
            )

    def get_reference_positions(
        self
        ) -> pd.DataFrame:
        """
        Get the reference position.
        """
        ## get the table with reference position and reference nucleotide
        df5=(self._obj
            .loc[:,['reference position','reference']]
            .dropna()
            .drop_duplicates()
            .sort_values('reference position')
            .reset_index(drop=True)
            )
        ## validate that the positions are continuous
        assert len(df5)==(df5['reference position'].max()-df5['reference position'].min()+1)
        return df5

    def get_counts_per_codon_mutation(
        self,
        start: int=None,
        ) -> pd.DataFrame:
        """
        Get counts of the codon mutations.

        Parameters:
            start (int): start position.
        """
        df5=self.get_reference_positions()
        if start is None:
            start=df5['reference position'].min()

        import math
        df5=(df5
            ## codon positions (assuming codons start from the 1st base)
            .assign(
            **{
                'codon position':lambda df: df['reference position'].sub(start).div(3).add(0.1).apply(math.ceil),
                'codon reference': lambda df: df.groupby('codon position')['reference'].transform(lambda x: ''.join(x)),
            }
            )
             ## merge-back the data
             .merge(
                 right=self._obj,
                 how='inner',
                 on=['reference position','reference'],
                 validate="1:m",
             )
        )

        df6=(
            df5
            .query("~(`CIGAR`.str.contains('I') or `CIGAR`.str.contains('D'))") ## remove indels
            .groupby(['query id','codon position']).filter(lambda df: len(df)==3)
            .assign(
            **{
                'codon read': lambda df: df.groupby(['query id','codon position'])['read'].transform(lambda x: ''.join(x)),
            }
            )
        )

        df7=(df6
            .query("~(`CIGAR`.str.contains('I') or `CIGAR`.str.contains('D'))")   ## remove indels
            .groupby(['reference position',"codon reference","codon read"]).size()## calculate depth
            .to_frame('depth').reset_index()
            .pipe(mask_wt,'depth',["codon reference","codon read"])               ## mask wt
            )
        return df7

    def get_counts_per_aa_mutation(
        self,
        to_aa: dict=None,
        ) -> pd.DataFrame:
        """
        Get counts of the amino acid mutations.

        Parameters:
            to_aa (dict): codon to amino acid map. defaults to the standard codon table.
        """
        df7=self.get_counts_per_codon_mutation()
        if to_aa is None:
            to_aa={'TCA': 'S', 'TCC': 'S', 'TCG': 'S', 'TCT': 'S', 'TTC': 'F', 'TTT': 'F', 'TTA': 'L', 'TTG': 'L', 'TAC': 'Y', 'TAT': 'Y', 'TAA': '*', 'TAG': '*', 'TGC': 'C', 'TGT': 'C', 'TGA': '*', 'TGG': 'W', 'CTA': 'L', 'CTC': 'L', 'CTG': 'L', 'CTT': 'L', 'CCA': 'P', 'CCC': 'P', 'CCG': 'P', 'CCT': 'P', 'CAC': 'H', 'CAT': 'H', 'CAA': 'Q', 'CAG': 'Q', 'CGA': 'R', 'CGC': 'R', 'CGG': 'R', 'CGT': 'R', 'ATA': 'I', 'ATC': 'I', 'ATT': 'I', 'ATG': 'M', 'ACA': 'T', 'ACC': 'T', 'ACG': 'T', 'ACT': 'T', 'AAC': 'N', 'AAT': 'N', 'AAA': 'K', 'AAG': 'K', 'AGC': 'S', 'AGT': 'S', 'AGA': 'R', 'AGG': 'R', 'GTA': 'V', 'GTC': 'V', 'GTG': 'V', 'GTT': 'V', 'GCA': 'A', 'GCC': 'A', 'GCG': 'A', 'GCT': 'A', 'GAC': 'D', 'GAT': 'D', 'GAA': 'E', 'GAG': 'E', 'GGA': 'G', 'GGC': 'G', 'GGG': 'G', 'GGT': 'G'}
            logging.info('standard codon table is used.')

        df8=(df7
        .assign(
            **{
                'aa reference':lambda df: df['codon reference'].map(to_aa),
                'aa read':lambda df: df['codon read'].map(to_aa),
            }
        )
        )
        return df8

    def get_frequencies_per_nt_mutation(
        self,
        ) -> pd.DataFrame:
        """
        Calculate the frequencies of the mutations.
        """
        df3=self.get_counts_per_nt_mutation()
        df2=self.get_coverage()
        return (
            df3
            .merge(                                    ## map the cumulative read depth
                right=df2,
                how='inner',
                on=['reference position','reference'],
                validate="m:1",
                suffixes=[' mutation',' total']
            )
            .assign(                                   ## calculate the frequency
                frequency=lambda df: df['depth mutation']/df['depth total']
            )
            .pipe(mask_wt,'frequency')                 ## mask wt
            )
    def get_map(
        self,
        value: str,
        aggfunc: str,
        fmt: str='{:.0f}',
        include_position=True,
        verbose: bool=False,
        ) -> pd.DataFrame:
        """
        Mutation map by the position of the mutation.

        Parameters:
            value (str): column with the values.
            aggfunc (str): the name of or the function to be used for aggregating values.
            fmt (str)': format of numbers displayed as output. defaults to '{:.0f}'.
            include_position (str): include the positions. defaults to True.
            verbose (bool): verbose. defaults to False.

        Returns:
            pd.DataFrame:
        """
        if 'aa reference' in self._obj:
            prefix='aa '
        elif 'codon reference' in self._obj:
            prefix='codon '
        elif 'reference' in self._obj:
            prefix=''
        else:
            raise ValueError(self._obj.columns.tolist())
        if verbose: logging.info(prefix)

        return (
            self._obj
            .groupby((['reference position'] if include_position else [])+[f'{prefix}reference',f'{prefix}read'])[value].agg('sum') ## aggregate depth
            .unstack([0,1] if include_position else [0])
            .sort_index(axis=1,level=0).fillna(0) ## formatting
            .style.format(fmt).background_gradient(cmap ='Blues_r',axis=None) ## color the table
        )

    def get_submap(
        self,
        value: str,
        aggfunc: str,
        fmt: str='{:.0f}',
        ) -> pd.DataFrame:
        """
        Substitution map.

        Parameters:
            value (str): column with the values.
            aggfunc (str): the name of or the function to be used for aggregating values.
            fmt (str)': format of numbers displayed as output. defaults to '{:.0f}'.
            include_position (str): include the positions. defaults to False.
            verbose (bool): verbose. defaults to False.

        Returns:
            pd.DataFrame:
        """
        return self.get_map(
            value=value,
            aggfunc=aggfunc,
            fmt=fmt,
            include_position=False,
            )

## for testing
def set_wd():
    import os
    print(">>>>>>>>>>>>>>>>> current dir:"+os.getcwd())
    if not os.getcwd().endswith("/examples"):
        if os.getcwd().endswith("/nbs"):
            os.chdir('../examples/')
        else:
            os.chdir('./examples/')
        print(">>>>>>>>>>>>>>>>> current dir:"+os.getcwd())

## Tests

In [ ]:
#|hide
## download the Demo Data
set_wd()
bam_url="http://ftp.sra.ebi.ac.uk/vol1/run/ERR346/ERR3461488/Lib052_0626_on_target.bam"
bam_path='inputs/test.bam'
# Reference: Rationally engineered Staphylococcus aureus Cas9 nucleases with high genome-wide specificity. https://www.ncbi.nlm.nih.gov/sra/?term=ERR3461488

## download in ./data folder
from pathlib import Path
if not Path(bam_path).exists():
    Path(bam_path).parent.mkdir(exist_ok=True)
    import urllib.request
    _=urllib.request.urlretrieve(
        bam_url,
        bam_path,
        )

### Read the bam file as a pandas dataframe

In [ ]:
%time
df=to_df(
    bam_path='inputs/test.bam', # path to the bam file
    contig='14',              # i.e. reference name/ target
    start=74846395,           # start position
    end=74846545,             # end position
    verbose=True,             # verbose
)
df.head(1)

CPU times: user 1e+03 ns, sys: 1 µs, total: 2 µs
Wall time: 4.77 µs


,query id,CIGAR,read position,reference position,reference,read
0,M04377:211:000000000-D637H:1:1101:7912:5949::u...,151M,0.0,74846395.0,C,C


In [ ]:
print(df.shape)
assert df.shape == (56424, 6)

(56424, 6)


### Coverage per position (`QC`)

In [ ]:
%time
data=df.dseq.get_coverage()
data.head(1)

CPU times: user 2 µs, sys: 1 µs, total: 3 µs
Wall time: 4.53 µs


,reference position,reference,depth
0,74846395.0,C,377


In [ ]:
print(data.shape)
assert data.shape == (151, 3)

(151, 3)


### Number of reads per mutations (`QC`)

In [ ]:
%time
data=df.dseq.get_counts_per_mutation()
data.head(1)

CPU times: user 9 µs, sys: 0 ns, total: 9 µs
Wall time: 15.7 µs


4    1
dtype: int64

In [ ]:
print(data.shape)
assert data.shape == (5,)

(5,)


### Read depth per nucleotide mutations

In [ ]:
%time
data=df.dseq.get_counts_per_nt_mutation()
data.head(1)

CPU times: user 2 µs, sys: 1 µs, total: 3 µs
Wall time: 4.29 µs


,reference position,reference,read,depth
0,74846395.0,C,C,NaN


In [ ]:
print(data.shape)
assert data.shape == (212, 4)

(212, 4)


### Read depth per codons

In [ ]:
%time
data=df.dseq.get_counts_per_codon_mutation()
data.head(1)

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 4.05 µs


,reference position,codon reference,codon read,depth
0,74846395.0,CCT,CCC,2.0


In [ ]:
print(data.shape)
assert data.shape == (330, 4)

(330, 4)


### Read depth per amino acids

In [ ]:
%time
data=df.dseq.get_counts_per_aa_mutation()
data.head(1)

CPU times: user 7 µs, sys: 1 µs, total: 8 µs
Wall time: 13.6 µs


,reference position,codon reference,codon read,depth,aa reference,aa read
0,74846395.0,CCT,CCC,2.0,P,P


In [ ]:
print(data.shape)
assert data.shape == (330, 6)

(330, 6)


### Normalize read depth of a mutations by the cumulative read depth, to calculate mutation freqency

In [ ]:
%time
data=df.dseq.get_frequencies_per_nt_mutation()
data.head(1)

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 4.05 µs


,reference position,reference,read,depth mutation,depth total,frequency
0,74846395.0,C,C,NaN,377,NaN


In [ ]:
print(data.shape)
assert data.shape == (212, 6)

(212, 6)


### Extract indels

In [ ]:
%time
data=(df
    .query("`CIGAR`.str.contains('I') or `CIGAR`.str.contains('D')") ## keep indels
    )
data.head(1)

CPU times: user 7 µs, sys: 0 ns, total: 7 µs
Wall time: 14.8 µs


,query id,CIGAR,read position,reference position,reference,read
37,M04377:211:000000000-D637H:1:1101:10676:7283::...,22M1D129M,36.0,74846395.0,C,C


In [ ]:
print(data.shape)
assert data.shape == (2699, 6)

(2699, 6)


In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()